In [ ]:
import pandas as pd
import decimal
import numpy as np
from datetime import datetime

# GGPT Data

In [ ]:
# Downloaded and compiled copy to use
# Note: if you edit the GGPT you'll have to re-download and compile to run this code
ggptCopy = r'C:\Users\Sarah\Documents\GEM\code\GGPT-Compile\2021_10_15_compiled\Global Gas Plant Tracker (GGPT) completed 2021-10-15.xlsx'

# EIA Data

## Initalize

### Variables

In [ ]:
stamp = datetime.now().strftime("%m %d %Y %H-%M-%S")

# Files downloaded
file860M = r'C:\Users\Sarah\Documents\GEM\research\North America\US\EIA\july_generator2021.xlsx'
# 860 spreadsheets, single tab
plant860 = r'C:\Users\Sarah\Documents\GEM\research\North America\US\EIA\eia8602020\2___Plant_Y2020.xlsx'
owner860 = r'C:\Users\Sarah\Documents\GEM\research\North America\US\EIA\eia8602020\4___Owner_Y2020.xlsx'
generator860 = r"C:\Users\Sarah\Documents\GEM\research\North America\US\EIA\eia8602020\3_1_Generator_Y2020.xlsx"

# see skiprows and skipfooter
eiaDropHead = [0, 1]

# Allowed technolgoy codes
allowedTechCodes = ['ST','GT','CC']

# Combined cycle is not operable but keep all units (as of July 2021 version)
# Make strings because convert Plant ID from number later in the code when building global ID
CT = ['10789', '52132', '55372', '55470', '10745', '57953', '54096', '50118']

# Reset these statuses to construction
setFixPlant = ['613', '10554']
setFixUnit = ['DBCC','CC_set']

# Columns we don't need for GGPT (will drop later, putting here for visibility)
eiaDropCols = ['Sector', 'Net Summer Capacity (MW)', 'Net Winter Capacity (MW)', 'Technology', 'Operating Month', 'Planned Retirement Month', 'Planned Operation Month',
'Google Map', 'Bing Map', 'Balancing Authority Code', 'Planned Derate Year', 'Planned Derate Month', 'Planned Derate of Summer Capacity (MW)', 'Planned Uprate Year', 'Planned Uprate Month', 
'Planned Uprate of Summer Capacity (MW)']

# EIA list of tabs to merge
# eiaTabs = ['Operating', 'Planned', 'Retired', 'Canceled or Postponed', 'Operating_PR', 'Planned_PR', 'Retired_PR']
eiaTabs = ['Operating', 'Planned', 'Operating_PR', 'Planned_PR']

eiaColumns = []

eiaList = [] # for storing each tab to merge later, better performance and this is a large table)
eiaLenQC = 0

#
runningRowCount= 0

#
generatorTabs = ['Operable', 'Proposed', 'Retired and Canceled']

### Read 860M data

In [ ]:
# Loop over each sheet to read in and merge
for i in eiaTabs:
    print("Processing tab: ", i)

    # Generator ID reads a string, preserves leading zeros in IDs, but Mason's code stripped it
    # Corrected to keep zeroes because there's plants with units '001' and '0001' 
    eia_ = pd.read_excel(file860M, sheet_name=i, skiprows=eiaDropHead, skipfooter=2, keep_default_na=False)
    eiaColumns += [x for x in eia_.columns if x not in eiaColumns]

    eiaLenQC += len(eia_.index) # Can compare this number later to the sum of all rows in the EIA Excel sheet

    # Keep only gas-fired plants + associaited steam turbines
    # XXX Need to wait until join in ALL fuels from 860 Generator data, this table has only first fuel
    # eia_ = eia_.loc[eia_['Energy Source Code'].isin(['NG','OTH'])]
   
    # Add status column based on the tab's name, EIA Status column inconsistent
    eia_['Status_Tab'] = i.lower()
            
    # Store this tab's data to merge
    eiaList += [eia_]

### Backfill columns that are missing in each tab

Columns not in other tabs will backfill as NaN, and worse, for integer columns
will backfill as a NaN float because NaN can't be an integer (e.g. Retired Year column).
Can't blanket apply .fillna() because some columns are string and other float, throws error.
And if leave float NaN, when try to create Sets later on for merging rows, they don't reduce
(float NaN have some kind of differentiating ID that disallows counting as duplicates)

In [ ]:
for i in eiaList:
    # Columns
    for col in eiaColumns:
        if col not in i.columns:
            i[col] = 'not found'

colLen = set()
for i in eiaList:
    # print(len(list(i.columns)))
    colLen.add(len(list(i.columns)))
if len(colLen) != 1:
    print('Error in columns.')
else:
    print('Columns complete.')

### Combine tabs

In [ ]:
# Combine tabs
eia = pd.concat(eiaList)

# Drop columns
eia.drop(columns=eiaDropCols, inplace=True)

# Total rows
print('\n')
print("Total row count of all tabs: ", eiaLenQC, '\n')

### Store 860M retirement data separately

In [ ]:
# Mason didn't put retired or canceled units in the original push so adding them now creates a lot of errors in table comparison
retired860M = pd.read_excel(file860M, sheet_name='Retired', skiprows=eiaDropHead, skipfooter=2, keep_default_na=False)
shelved860M = pd.read_excel(file860M, sheet_name='Canceled or Postponed', skiprows=eiaDropHead, skipfooter=2, keep_default_na=False)

# TODO: see what's in GGPT and add in now so formats correctly
# Otherwise CC units won't match also

### Read 860 Data

In [ ]:
# 860 spreadsheets, single tab
plantData = pd.read_excel(plant860, skiprows=[0], keep_default_na=False)
ownerData = pd.read_excel(owner860, skiprows=[0], keep_default_na=False)

# Loop over each sheet to read in and merge
g_=[]
for i in generatorTabs:
    print("Processing tab: ", i)
    if i in ['Operable', 'Proposed']:
        gData = pd.read_excel(generator860, sheet_name=i, skiprows=[0], skipfooter=1, keep_default_na=False)
    else:
        gData = pd.read_excel(generator860, sheet_name=i, skiprows=[0], keep_default_na=False)
    
    gCol = [x for x in list(gData.columns) if x not in ['Generator ID', 'Plant Code', 'Energy Source 1', 'Energy Source 2', 'Energy Source 3', 'Energy Source 4', 'Energy Source 5', 'Energy Source 6', 
    'Associated with Combined Heat and Power System']]
    gData.drop(columns=gCol, inplace=True)

    # Keep ~only gas-fired plants
    # gIndex = gData.loc[~gData['Plant Code'].isin(list(eia['Plant ID']))].index
    # gData.drop(index=gIndex, inplace=True)
            
    # Store this tab's data to merge
    g_ += [gData]

# Combine tabs
generatorData = pd.concat(g_)

## Add 860 data

### 860 Fuels + CHP
### (fuel has to be before Generator IDs become lists or else can't merge easily)

In [ ]:
# EIA fuel columns
fuelCols = ['Energy Source 1', 'Energy Source 2', 'Energy Source 3', 'Energy Source 4', 'Energy Source 5', 'Energy Source 6']

# Dictionary of some EIA fuel codes to GGPT domain
fuelMix = {'OBS':'B',
'WDS':'B',
'BFG':'BFG',
'OBL':'BL',
'WDL':'BL',
'ANT':'C',
'BIT':'C',
'LIG':'C',
'SGC':'C',
'SUB':'C',
'WC':'C',
'RC':'C',
'DFO':'FO',
'RFO':'FO',
'JF':'J',
'SUN':'S',
'SGP':'SG'}

# Redo fuel codes
for i in fuelCols:
   generatorData[i] = generatorData[i].apply(lambda x: fuelMix[x] if (x in fuelMix.keys()) else x)

In [ ]:
# Merge fuels into one column
# First, in case of duplicates, make a set
generatorData['fuelMerge'] = generatorData[fuelCols].values.tolist()
generatorData['fuelMerge'] = generatorData['fuelMerge'].apply(lambda x: list(set([x for x in x if  x != ''])))
generatorData['fuelMerge'] = generatorData['fuelMerge'].apply(lambda x: '/'.join(x))

# multi-column matching issue, so made single index; can't '+' if there's hidden strings or ints in the column
generatorData = generatorData.astype({'Plant Code':'str', 'Generator ID': 'str'})
generatorData['gid'] = generatorData['Plant Code'] + "_" + generatorData['Generator ID']

eia = eia.astype({'Plant ID':'str', 'Generator ID': 'str'})
# eia['gMix'] = list(eia['Plant ID'] + "_" + eia['Generator ID'])
eia['gid'] = eia['Plant ID'] + '_' + eia['Generator ID'] 

rmGenCols = fuelCols + ['Plant Code', 'Generator ID']
g2 = generatorData.drop(columns=rmGenCols)
g2.rename(columns={"Associated with Combined Heat and Power System": 'CHP'}, inplace=True)

# merge
eia1 = eia.merge(g2, how='left', on='gid')

In [ ]:
# Fix unmatched CHP values, if any (in July 2021 it was 154)
print("This many CHP errors: ", len(eia1.loc[eia1['CHP'].isna()].index))
eia1.loc[eia1['CHP'].isna(), "CHP"] = 'not found'

# Compare Energy Code column to fuelMerge and replace where needed
# For units with just "NG", but that are actually multi-fuel, will add dother fuels in 
print("This many Fuel replacements: ", len(eia1.loc[(eia1['Energy Source Code'] != eia1['fuelMerge']) & (~eia1['fuelMerge'].isna()), 'Energy Source Code']))
eia1.loc[(eia1['Energy Source Code'] != eia1['fuelMerge']) & (~eia1['fuelMerge'].isna()), 'Energy Source Code'] = eia1.loc[(eia1['Energy Source Code'] != eia1['fuelMerge']) & (~eia1['fuelMerge'].isna()), 'fuelMerge']

# Drop extra columns after merge
eia1 = eia1.drop(columns=['fuelMerge'])

# Now drop rows that don't contain NG or that one steam turbine that's 'other' fuel
# Also IF ever do this on fuelMegre column note to keep anything anything NaN, that happened for plants in Puerto Rico missing from Generators
dropFuel = eia1.loc[~eia1['Energy Source Code'].apply(lambda x: True if ('NG' in str(x) or str(x) == "OTH") else False)].index
eia1 = eia1.drop(index=dropFuel)


### Join in 860 data, ownership

In [ ]:
# Format
# Find plants missing values (returns as type str which causes issues adding % sign). 

# Find empty percentage rows
ownerNotFound = ownerData.loc[ownerData['Percent Owned'].apply(lambda x: True if (x == ' ') else False)].index
# If percent owned is empty put as 0 for now so can do math
ownerData.loc[ownerData.index.isin(ownerNotFound), 'Percent Owned'] = 0
# Coerce into right formatting
ownerData['Percent Owned'] = ownerData['Percent Owned'].astype(float)
ownerData['Percent Owned'] = ownerData['Percent Owned'] * 100
ownerData['Percent Owned'] = ownerData['Percent Owned'].astype(int) # remove decimal points
ownerData['Percent Owned'] = ownerData['Percent Owned'].astype(str)
# Now that its string, turn 0 from the blanks into 'not found'
ownerData.loc[ownerData.index.isin(ownerNotFound), 'Percent Owned'] = '[% not found]'
# TODO: recalcuating % multiple owners per unit (Foster Wheeler Martinez Inc [99%]; BRE Partner - BRE [1%]; Foster Wheeler Power Sys Inc [99%]; BRE Partner - BRE [1%])
# TODO if one, remove 100%
# Make into GGPT format
ownerData['Owner Name'] = ownerData['Owner Name'].astype(str)
ownerData['oID'] = ownerData['Owner Name'] + ' [' + ownerData['Percent Owned'] + '%]'

# Group by Generator ID
# since one generator can have multiple owners this keeps there from being multiple GID matchs on eia1 merge
ownerData = ownerData.groupby(['Plant Code', 'Generator ID'])['oID'].apply('; '.join).reset_index()

# Global ID - can't group on Generator ID from owner table because retired units will mess up matching codes
ownerData['Plant Code'] = ownerData['Plant Code'].astype(str)
ownerData['Generator ID'] = ownerData['Generator ID'].astype(str)
ownerData['gid'] = ownerData['Plant Code'] + '_' + ownerData['Generator ID'] 

print(len(eia1.index))
print(len(eia1[['gid']].drop_duplicates().index))

# [can't Merge on Utility ID <--> Entity ID, eia data only puts first entity code, not a list to match]
eiaMultiOwner = eia1.merge(right=ownerData[['gid', 'oID']], how='left', on='gid')
print(eiaMultiOwner.columns.values)

In [ ]:
# If not matched in Ownership table, the entity from EIA 860M is the owner
# TODO: plants that only partially match in Owners table, if 100% then need to do math for Entity portion ownership and re-write
# TODO: might be worth not forming full text string until after that
sameOwner = eiaMultiOwner.loc[eiaMultiOwner['oID'].isna()]
print('sameowner', len(sameOwner.index))
sameOwner = sameOwner.drop(columns=['oID'])
sameOwner['oID']  = sameOwner['Entity Name'] # Note: don't include 100%, we don't do that for sponsors and won't match
print('eia1',len(eia1.index))
dSO = eiaMultiOwner.loc[eiaMultiOwner['gid'].isin(sameOwner['gid'].tolist())].index.tolist()
print('dso',len(dSO))
print('owner1',len(eiaMultiOwner.index))
newEIA = eiaMultiOwner.drop(index=dSO)
print(len(newEIA.index))
newEIA = newEIA.append(other=sameOwner, ignore_index=True)
print(len(newEIA.index))
newEIA = newEIA.drop(columns=['gid', 'Entity ID', 'Entity Name'])
newEIA = newEIA.rename(columns={'oID':'Ownership'})

## Merge individual combined cycle units into single sets ##

### Groupby "Unit Code", the identifier for common components of a combined cycle set

In [ ]:
# Merge units into proper sets
# 'ST', 'GT', and 'CS' are individual. 'CA' (steam) and 'CT' (gas turbine) need to be combined to 'CC'
ccSets = newEIA.loc[newEIA['Prime Mover Code'].isin(['CA', 'CT'])]
print('CC to combine: ', len(ccSets.index))

#Deal with these units separately
print('CC units with special status issue to resolve later: ', CT)
ccSets = ccSets.drop(index=ccSets.loc[ccSets['Plant ID'].isin(CT)].index)
ccToRemove = ccSets.index

runningRowCount = 0 - len(ccToRemove)
print('Running row count: ', runningRowCount)

# Group by plant ID and Unit Code fields
# Sum capacities, group Generator IDs, keep sets of values that could possibly vary between units
ccGroup = ccSets.groupby(['Plant ID', 'Unit Code', 'Status_Tab'], as_index=False).agg({'Nameplate Capacity (MW)': 'sum', 'Generator ID': lambda x: list(x), 'Status': lambda x: set(x),
'Operating Year': lambda x: set(x), 'Planned Retirement Year': lambda x: set(x), 'Planned Operation Year': lambda x: set(x), 'CHP': lambda x: set(x), 'Ownership': lambda x: set(x)})

# Set the correct tech code
ccGroup['Prime Mover Code'] = 'CC'

# IF not unit code, mason called it 'cc_set'
ccGroup.loc[ccGroup['Unit Code'] == '', 'Unit Code'] = 'CC_set'

### Deal with conflicting values from merged rows, such as start year

In [ ]:
# Check Sets for multiple values and turn into a range if necessary (e.g. different start years for units of a CC generator)
checkSets = ['Status', 'Operating Year', 'Planned Retirement Year', 'Planned Operation Year', 'CHP', 'Ownership']
for i in checkSets:
    # Wherever there are multiple values print them to screen
    print('\n', i)
    print("Cells with multiple values: ", len(ccGroup.loc[ccGroup[i].apply(len) != 1].index))
    # Possible issues combing NaN and year
    if len(ccGroup.loc[(ccGroup[i].apply(lambda x: 'not found' in list(x))) & (ccGroup[i].apply(len) > 1)].index) > 0:
        print('ERROR Values and not founds: ', len(ccGroup.loc[(ccGroup[i].apply(lambda x: 'not found' in list(x))) & (ccGroup[i].apply(len) > 1)].index))
    # For mutliple years per new CC value, create year range
    check = []
    check = ccGroup.loc[ccGroup[i].apply(len) != 1]
    ownerCheck = ccGroup.loc[ccGroup['Ownership'].apply(lambda x: True if (type(x) == set and len(list(x)) > 1) else False)].index
    if len(check.index) > 0 and 'Year' in i:
        ccGroup.loc[ccGroup[i].apply(len) != 1, i] = ccGroup.loc[ccGroup[i].apply(len) != 1][i].apply(lambda x: str(int(min(list(x)))) + '-' +  str(int(max(list(x)))) )
    elif len(check.index) > 0 and i == 'CHP':
        ccGroup.loc[(ccGroup[i].apply(len) != 1) & ccGroup[i] != 'not found', i] = "Y"
    elif len(ownerCheck) > 1 and i == 'Ownership':
         #TODO: make a set, remove duplicates, while adding their percentages
        ccGroup.loc[ownerCheck, 'Ownership'] = ccGroup.loc[ownerCheck, 'Ownership'].apply('; '.join)
    else:
        print("Manually fix: ", i)
        # print(ccGroup.loc[ccGroup[i].apply(len) != 1])

    # Wherever there's just one value, replace the set with the value
    if i == 'Ownership':
        ccGroup.loc[~ccGroup.index.isin(ownerCheck), i] = ccGroup.loc[~ccGroup.index.isin(ownerCheck)][i].apply(''.join)
    else:
        ccGroup.loc[ccGroup[i].apply(len) == 1, i] = ccGroup.loc[ccGroup[i].apply(len) == 1][i].apply(lambda x: list(x)[0])

In [ ]:
# Join back relevant plant-level columns
eiaBackfill = newEIA.drop_duplicates(subset='Plant ID', ignore_index=True)
eiaBackfill = eiaBackfill.drop(columns=['Unit Code', 'Nameplate Capacity (MW)', 'Generator ID', 'Status', 'Operating Year', 'Planned Retirement Year', 'Planned Operation Year',
 'Prime Mover Code', 'Status_Tab', 'CHP', 'Ownership'])
ccGroupJoin = ccGroup.merge(eiaBackfill, on='Plant ID', how='left')

In [ ]:
# Remove units from eia dataset, add back-in the new sets
eiaFormatted = newEIA.drop(index=ccToRemove)

# Check to make sure columns match first
if sorted(list(eiaFormatted.columns)) != sorted(list(ccGroupJoin.columns)):
    print("Error, missing columns")
    exit()

# Append CC rows
eiaFormatted = eiaFormatted.append(ccGroupJoin, ignore_index=True)

# QC
runningRowCount = runningRowCount + len(ccGroupJoin.index)
print('Running row count: ', runningRowCount)

### Combined cycle fixes

In [ ]:
#TODO: double check when split CC units out, concat unit code w gen code '-'

In [ ]:
# Manual fixes
# For single units in standby within CC unit
# convert tech to just GT or ST and make mothballed
# CT variable is taken from manually reviewing the rows from cc group with multiple statuses

# Unit ids
CT_U = list(eiaFormatted.loc[(eiaFormatted['Plant ID'].isin(CT)) & eiaFormatted['Prime Mover Code'].isin(['CT','CA'])].index)

# Deal with mothballed generators
sepCode = ['\(SB\)', '\(OS\)', '\(OA\)']
ccSetsSkip = eiaFormatted.loc[(eiaFormatted['Status'].str.contains('|'.join(sepCode))) & (eiaFormatted['Plant ID'].isin(CT)) & eiaFormatted['Prime Mover Code'].isin(['CT','CA'])]
CT_2 = [x for x in CT_U if x not in list(ccSetsSkip.index)]

for x in list(ccSetsSkip.index):
    eiaFormatted.loc[x, 'Status'] = 'mothballed'
    if eiaFormatted.loc[x, 'Prime Mover Code'] == 'CT':
        eiaFormatted.loc[x, 'Prime Mover Code'] = 'GT'
    else:
        eiaFormatted.loc[x, 'Prime Mover Code'] = 'ST'

In [ ]:
# Now see if there's still a CC unit with one status to combine
# Merge units into proper sets
# 'ST', 'GT', and 'CS' are individual. 'CA' (steam) and 'CT' (gas turbine) need to be combined to 'CC'
ccSetsM = eiaFormatted.loc[eiaFormatted.index.isin(CT_2)]
ccToRemoveM = ccSetsM.index

# QC
runningRowCount = runningRowCount - len(ccToRemoveM)
print('Running row count: ', runningRowCount)

# Group by plant ID and Unit Code fields
# Sum capacities, group Generator IDs, keep sets of values that could possibly vary between units
ccGroupM = ccSetsM.groupby(['Plant ID', 'Unit Code', 'Status_Tab'], as_index=False).agg({'Prime Mover Code': lambda x: set(x), 'Nameplate Capacity (MW)': 'sum', 'Generator ID': lambda x: list(x), 'Status': lambda x: set(x),
'Operating Year': lambda x: set(x), 'Planned Retirement Year': lambda x: set(x), 'Planned Operation Year': lambda x: set(x), 'CHP': lambda x: set(x), 'Ownership': lambda x: set(x)})

## QC ##
# Check for multiple statuses
if len(ccGroupM.loc[ccGroupM['Status'].apply(len) != 1].index) > 0:
    print("Cells with multiple Status values: ", ccGroupM.loc[ccGroupM['Status'].apply(len) != 1])

## Sort out what is still CC and what's individual turbines now ##
# Check is a CC unit still
ccGroupM.loc[ccGroupM['Prime Mover Code'].apply(len) > 1, 'Prime Mover Code'] = 'CC'

# If not, remove from the groupby and re-insert as individual units
cgmi = ccGroupM.loc[ccGroupM['Prime Mover Code'].apply(len) == 1].index
ccGroupM.drop(index=cgmi, inplace=True)

# Create single list of generator ids to drop
gm = ccGroupM['Generator ID'].apply(lambda x: x if (type(x) == list) else [x])
gmDrop = np.concatenate(gm.tolist())
# drop
ccSetsM2 = ccSetsM.drop(index=ccSetsM.loc[ccSetsM['Generator ID'].isin(gmDrop)].index)
# Rename codes
ccSetsM2.loc[ccSetsM2['Prime Mover Code'] == 'CT', 'Prime Mover Code'] = 'GT'
ccSetsM2.loc[ccSetsM2['Prime Mover Code'] == 'CA', 'Prime Mover Code'] = 'ST'




In [ ]:
# Check Sets for multiple values and turn into a range if necessary (e.g. different start years for units of a CC generator)
for i in checkSets: # <-- previously defined
    print('\n', i)
    # Possible issues combing NaN and year
    if len(ccGroupM.loc[(ccGroupM[i].apply(lambda x: 'not found' in list(x))) & (ccGroupM[i].apply(len) > 1)].index) > 0:
        print('ERROR Values and not founds: ', len(ccGroupM.loc[(ccGroupM[i].apply(lambda x: 'not found' in list(x))) & (ccGroupM[i].apply(len) > 1)].index))
   
    # For mutliple years per new CC value, create year range
    if 'Year' in i:
        ccGroupM.loc[ccGroupM[i].apply(len) != 1, i] = ccGroupM.loc[ccGroupM[i].apply(len) != 1][i].apply(lambda x: str(int(min(list(x)))) + '-' +  str(int(max(list(x)))) )
    
    check = ccGroupM.loc[ccGroupM[i].apply(len) != 1]
    ownerCheck = ccGroupM.loc[ccGroupM['Ownership'].apply(lambda x: True if (type(x) == set and len(list(x)) > 1) else False)].index
    if len(check.index) > 0 and i == 'CHP':
        ccGroupM.loc[(ccGroupM[i].apply(len) != 1) & ccGroupM[i] != 'not found', i] = "Y"
    elif len(ownerCheck) > 1 and i == 'Ownership':
         #TODO: make a set, remove duplicates, while adding their percentages
        ccGroupM.loc[ownerCheck, 'Ownership'] = ccGroupM.loc[ownerCheck, 'Ownership'].apply('; '.join)
    # Wherever there's just one value, replace the set with the value
    if i != 'Ownership':
        ccGroupM.loc[ccGroupM[i].apply(len) == 1, i] = ccGroupM.loc[ccGroupM[i].apply(len) == 1][i].apply(lambda x: list(x)[0])
    else:
        ccGroupM.loc[~ccGroupM.index.isin(ownerCheck), i] = ccGroupM.loc[~ccGroupM.index.isin(ownerCheck)][i].apply(''.join)

## Merge all CC-type units after editing ##
# Join back relevant plant-level columns before append
ccGroupM = ccGroupM.merge(eiaBackfill, on='Plant ID', how='left')
# Check to make sure columns match first
if sorted(list(ccGroupM.columns)) != sorted(list(ccSetsM2.columns)):
    print("Error, missing columns")
    exit()
ccGroupM = ccGroupM.append(ccSetsM2, ignore_index=True)

In [ ]:
# Remove units from eia dataset, add back-in the new sets
eiaFormatted.drop(index=ccToRemoveM, inplace=True)

# Check to make sure columns match first
if sorted(list(eiaFormatted.columns)) != sorted(list(ccGroupM.columns)):
    print("Error, missing columns")
    exit()

# Append CC rows
eiaFormatted = eiaFormatted.append(ccGroupM, ignore_index=True)

#QC
runningRowCount = runningRowCount + len(ccGroupM.index)
print('Running row count: ', runningRowCount)

## Formatting

### Capacity cut-off

In [ ]:
# remove units with capacity issues if not operating/planned
tmp = pd.to_numeric(eiaFormatted['Nameplate Capacity (MW)'], errors='coerce')
print("Units missing capacity data or other errors: ", '\n', eiaFormatted.loc[tmp.isna(), ['Plant Name','Status_Tab']])

#QC
runningRowCount = runningRowCount - len(eiaFormatted.loc[tmp.isna()].index)
print('Running row count: ', runningRowCount)

eiaFormatted.drop(index=eiaFormatted.loc[tmp.isna()].index, inplace=True)

# remove units <50 MW
eiaFormatted['Nameplate Capacity (MW)'] = eiaFormatted['Nameplate Capacity (MW)'].apply(lambda x: decimal.Decimal(x).quantize(decimal.Decimal('1'), 
    rounding=decimal.ROUND_HALF_UP)) # Python 3 went to Round Half Even, overriding

#QC
runningRowCount = runningRowCount - len(eiaFormatted.loc[eiaFormatted['Nameplate Capacity (MW)'] < 50].index)
print('Running row count: ', runningRowCount)

eiaFormatted.drop(index=eiaFormatted.loc[eiaFormatted['Nameplate Capacity (MW)'] < 50].index, inplace=True)

### Technology codes

In [ ]:
# Change 'CS' to 'CC', etc
eiaFormatted.loc[eiaFormatted['Prime Mover Code'] == 'CS', 'Prime Mover Code'] = 'CC'

# Remove IC and other tech
print("Current tech codes: ", set(eiaFormatted['Prime Mover Code']))
#QC
runningRowCount = runningRowCount - len(eiaFormatted.loc[~eiaFormatted['Prime Mover Code'].isin(allowedTechCodes)].index)
print('Running row count: ', runningRowCount)

eiaFormatted.drop(eiaFormatted.loc[~eiaFormatted['Prime Mover Code'].isin(allowedTechCodes)].index, inplace=True)

### Join in 860 data, city

In [ ]:
# Use City column from plant data spreadsheet
pDrop = [x for x in plantData.columns.values if x not in ['Plant Code', 'City']]
plantData.drop(columns=pDrop, inplace=True)
plantData = plantData.astype({'Plant Code':str})
eiaFormatted = eiaFormatted.merge(plantData, how='left', left_on='Plant ID', right_on='Plant Code')
eiaFormatted.drop(columns=['Plant Code'], inplace=True)

# some plants missing from generators table (noticed in Puerto Rico) so backfill NaN
eiaFormatted.loc[eiaFormatted['City'].isna(), 'City'] = 'not found'

### Generator ID lists to string

In [ ]:
# Backfill single units with a unit name
eiaFormatted.loc[eiaFormatted['Unit Code'] == '', 'Unit Code'] = eiaFormatted.loc[eiaFormatted['Unit Code'] == '', 'Generator ID']

# Make string for CC units and Other IDs
eiaFormatted['Generator ID'] = eiaFormatted['Generator ID'].apply(lambda x: ', '.join(sorted(x)) if (type(x) == list) else x)

### EIA Status

In [ ]:
# Standardize status values
# statuses = list(eiaFormatted['Status'])
# s = []
# s += [x for x in statuses if type(x) != set]
# print(set(s))
st = {
    '(U) Under construction, less than or equal to 50 percent complete': 'construction',
    '(OP) Operating': 'operating', 
    '(OS) Out of service and NOT expected to return to service in next calendar year': 'mothballed', 
    '(OA) Out of service but expected to return to service in next calendar year': 'mothballed', 
    '(P) Planned for installation, but regulatory approvals not initiated': 'proposed', 
    '(TS) Construction complete, but not yet in commercial operation': 'construction', 
    '(T) Regulatory approvals received. Not under construction': 'proposed', 
    '(V) Under construction, more than 50 percent complete': 'construction', 
    '(SB) Standby/Backup: available for service but not normally used': 'mothballed', 
    '(L) Regulatory approvals pending. Not under construction': 'proposed'
}

for key in st:
    eiaFormatted.loc[eiaFormatted['Status'] == key, 'Status'] = st[key]

# Reset these statuses to construction
eiaFormatted.loc[(eiaFormatted['Plant ID'].isin(setFixPlant)) & eiaFormatted['Unit Code'].isin(setFixUnit), 'Status'] = 'construction'

# Pre-process
eiaFormatted['Status_Tab'] = eiaFormatted['Status_Tab'].apply(lambda x: x.replace('_pr', ''))

# Move "planned" tab's start year column
eiaFormatted.loc[eiaFormatted['Status_Tab'].isin(['planned']), 'Operating Year'] = eiaFormatted.loc[eiaFormatted['Status_Tab'].isin(['planned']), 'Planned Operation Year']
eiaFormatted.drop(columns=['Planned Operation Year'], inplace=True)

# Deal with tabs without status column
eiaFormatted.loc[(eiaFormatted['Status'] ==  'not found'), 'Status'] = eiaFormatted.loc[(eiaFormatted['Status'] ==  'not found'), 'Status_Tab']
eiaFormatted.loc[(eiaFormatted['Status'] ==  'canceled or postponed'), 'Status'] = 'cancelled'
eiaFormatted.drop(columns=['Status_Tab'], inplace=True)

### Other ID field values for GGPT

In [ ]:
# Create 'Other ID' fields to match GGPT
# TODO: match the quotes 'EIA':'5' etc
eiaFormatted['Other IDs (location)'] = eiaFormatted['Plant ID'].apply(lambda x: '{EIA: ' + str(x) + '}')
eiaFormatted['Other IDs (unit)'] = eiaFormatted['Generator ID'].apply(lambda x: '{EIA: ' + str(x) + '}')

### Columns where prefer blank to 'not found' for comparison later

In [ ]:
# eiaFormatted.loc[eiaFormatted['Retirement Year'] == 'not found','Retirement Year'] = ''
eiaFormatted.loc[eiaFormatted['Planned Retirement Year'] == 'not found','Planned Retirement Year'] = ''

### Find any sets left over, or won't sort values later

In [ ]:
# for i in eiaFormatted.columns:
#     print(eiaFormatted.loc[eiaFormatted[i].apply(lambda x: True if (type(x)==set) else False)])

# eiaFormatted.loc[eiaFormatted['Plant ID']==10554]

## Final QC

In [ ]:
print("Starting row count: ", len(eia1.index))

print("Ending row count: ", len(eiaFormatted.index))

print("Total removed rows: ", runningRowCount)

print("Adding removed and end count: ", len(eiaFormatted.index) + (-1*runningRowCount))

In [ ]:
eiaFormatted.to_excel(r'C:\Users\Sarah\Documents\GEM\code\GGPT-US-EIA\scratch\eiaFormatted_' + stamp + '.xlsx', index=False)

In [ ]:
# TODO: pygsheets
ggpt = pd.read_excel(ggptCopy, sheet_name='GGPT - Gas Units',
 keep_default_na=False)
ggpt = ggpt.loc[ggpt['Country'] == 'United States']

In [ ]:
eiaQC = eiaFormatted.copy(deep=True)
eiaQC['labelCheck'] = eiaQC['Plant ID'].apply(str) + " " + eiaQC['Generator ID'] # if same as GGPT field drops in compare function, hard to find plants
if len(eiaQC.loc[eiaQC.duplicated(subset='labelCheck')].index) > 0:
    print('EIA ID ERROR')
    eiaQC.loc[eiaQC.duplicated(subset='labelCheck')]

# GGPT
# example format {'EIA': '56249'}	{'EIA': '5'}
ggpt['Other IDs (location)'] = ggpt['Other IDs (location)'].str.replace(r"{'EIA': '", '', regex=True).str.replace(r"'}", '', regex=True)

ggpt['Other IDs (unit)'] = ggpt['Other IDs (unit)'].str.replace(r"{'EIA': '", '', regex=True).str.replace(r"'}", '', regex=True)
ggpt['Other IDs (unit)'] = ggpt['Other IDs (unit)'].apply(lambda x: x.split(', ') if (', ' in x) else x)
ggpt['Other IDs (unit)'] = ggpt['Other IDs (unit)'].apply(lambda x: sorted(x) if (type(x) == list) else x)
ggpt['Other IDs (unit)'] = ggpt['Other IDs (unit)'] .apply(lambda x: (', ').join(x) if (type(x) == list) else x)

ggpt['labelCheck'] = ggpt['Other IDs (location)'] + " " + ggpt['Other IDs (unit)']
if len(ggpt.loc[ggpt.duplicated(subset='labelCheck')].index) > 0:
    # If the label is blank because GGPT has no EIA data, skip
    badLabels = ggpt.loc[(ggpt.duplicated(subset='labelCheck'))]
    if len(badLabels.loc[~badLabels['labelCheck'].isin([' ', 'not found not found'])].index) > 0:
        print('GGPT ID ERROR')
        print(len(badLabels.loc[~badLabels['labelCheck'].isin([' ', 'not found not found'])].index))
        print(badLabels.loc[~badLabels['labelCheck'].isin([' ', 'not found not found'])])

In [ ]:
# eiaQC.loc[eiaQC['labelCheck'].apply(lambda x: True if ('61034' in x) else False)]
# eiaFormatted.loc[eiaFormatted['Plant ID'].apply(lambda x: True if ('61034' in x) else False)]

In [ ]:
# ggpt.loc[ggpt['labelCheck'].apply(lambda x: True if ('61034' in x) else False)]

### Column names to match GGPT

In [ ]:
# drop columns to compare to EIA
# Plant name has slight variations and states are in differentf ormat, so have to drop for now as well
refCols = [x for x in list(ggpt.columns) if '[ref]' in x]
ggpt.drop(columns=refCols, inplace=True)
ggpt.drop(columns=['Location accuracy', 'Major area (prefecture, district)',
'Parent', 'Sponsor LEI', 'Parent LEI', 'Parent HQ country',
'Last Updated', 'Country', 'GEM location ID', 'GEM unit ID', 'WEPP location ID', 'WEPP unit ID',
'Plant name (local script)', 'Other plant names', 'Wiki URL', 'Other IDs (location)','Other IDs (unit)','Region',
'Plant name', 'Subnational unit (province, state)'], inplace=True)
ggpt.reset_index(inplace=True, drop=True)

In [ ]:

# Save this before lose columns for writing excel later
eiaQCForPrinting = eiaQC.copy(deep=True)

# drop cols
eiaQC.drop(columns=['Plant ID', 'Generator ID', 'Other IDs (location)','Other IDs (unit)', 
'Plant State', 'Plant Name'], inplace=True)

eiaQC.reset_index(inplace=True, drop=True)

repCols = { 
  'Ownership': 'Sponsor',
  'Unit Code': 'Unit name',
   'Nameplate Capacity (MW)': 'Capacity elec. (MW)',
   'Prime Mover Code': 'Technology', 
   'Operating Year': 'Start year', 
   'Planned Retirement Year': 'Planned retire',
      'County': 'Local area (taluk, county)',
      'Energy Source Code': 'Fuel'
}
eiaQC.rename(columns=repCols, inplace=True)

eiaQC['Retired year'] = ''

In [ ]:
eiaQC = eiaQC.reindex(columns=sorted(eiaQC.columns))
ggpt = ggpt.reindex(columns=sorted(ggpt.columns))

In [ ]:
eiaQC.sort_values(by='labelCheck', ignore_index=True,  inplace=True)
ggpt.sort_values(by='labelCheck', ignore_index=True, inplace=True)

In [ ]:
# Crosscheck
missingInGGPT=[x for x in list(eiaQC['labelCheck']) if x not in list(ggpt['labelCheck'])]
print(len(missingInGGPT))
print(len(list(eiaQC['labelCheck'])))

# Possibilities: leading zero issue, CC unti split up, units combined, retired (see next cell), etc
missingInEIA=[x for x in list(ggpt['labelCheck']) if x not in list(eiaQC['labelCheck'])]

In [ ]:
# GGPT: Check if it's because in retired/shelved tab
retired860M['labelCheck'] = retired860M['Plant ID'].astype(str) + ' ' + retired860M['Generator ID'].astype(str)
shelved860M['labelCheck'] = shelved860M['Plant ID'].astype(str) + ' ' + shelved860M['Generator ID'].astype(str)
retiredGGPT = [x for x in missingInEIA if x in retired860M['labelCheck'].tolist()]
shelvedGGPT = [x for x in missingInEIA if x in shelved860M['labelCheck'].tolist()]

missingInEIA = [x for x in missingInEIA if x not in retiredGGPT]
missingInEIA = [x for x in missingInEIA if x not in shelvedGGPT]

In [ ]:
# EIA -  many of these are coal plants in GCPT that also use NG
# TODO: start exception list so don't have to think through this every time (i.e. store the IDs we checked already)
# Print from table with all columns so can understand better how missed in GGPT
# TODO: print so can easily paste into GGPT as-is
eiaQCForPrinting.loc[eiaQCForPrinting['labelCheck'].isin(missingInGGPT)].to_excel(r'C:\Users\Sarah\Documents\GEM\code\GGPT-US-EIA\scratch\eiaMissingFromGGPT_' + stamp + '.xlsx', index=False)

#GGPT
ggpt.loc[ggpt['labelCheck'].isin(missingInEIA)].to_excel(r'C:\Users\Sarah\Documents\GEM\code\GGPT-US-EIA\scratch\ggptMissingFromEIA_' + stamp + '.xlsx', index=False)
retired860M.loc[retired860M['labelCheck'].isin(retiredGGPT)].to_excel(r'C:\Users\Sarah\Documents\GEM\code\GGPT-US-EIA\scratch\ggptRetiredInEIA_' + stamp + '.xlsx', index=False)
shelved860M.loc[shelved860M['labelCheck'].isin(shelvedGGPT)].to_excel(r'C:\Users\Sarah\Documents\GEM\code\GGPT-US-EIA\scratch\ggptShelvedInEIA_' + stamp + '.xlsx', index=False)

In [ ]:
# For shared IDs, compare
eiaQC2 = eiaQC.copy(deep=True)

inGGPT = [x for x in list(eiaQC2['labelCheck']) if x in list(ggpt['labelCheck'])]
eiaQC2 = eiaQC2.loc[eiaQC2['labelCheck'].isin(inGGPT)]
eiaQC2.set_index(keys='labelCheck', inplace=True, drop=False)

ggpt2 = ggpt.copy(deep=True)
inQC2=[x for x in list(ggpt2['labelCheck']) if x in list(eiaQC2['labelCheck'])]
ggpt2 = ggpt2.loc[ggpt2['labelCheck'].isin(inQC2)]
ggpt2.set_index(keys='labelCheck', inplace=True, drop=False)

print(len(eiaQC2.index))
print(len(ggpt2.index))

In [ ]:
# Corece all Year values into strings or else dtype is compared and returns False on int/str matches
eiaQC2 = eiaQC2.astype({'Start year':'str', 'Retired year': 'str', 'Planned retire': 'str', 'Latitude': 'str', 'Longitude':'str'})
ggpt2 = ggpt2.astype({'Start year':'str', 'Retired year': 'str', 'Planned retire': 'str', 'Latitude': 'str', 'Longitude':'str'})

# print(eiaQC2.dtypes)
# print(ggpt2.dtypes)

In [ ]:
comparison = eiaQC2.compare(other=ggpt2, keep_shape=False)
multiCol=list(comparison.columns)
t2 = [x[0]+x[1].replace('self', 'eia') for x in multiCol]

c2 = comparison.droplevel(level=[0], axis=1)

c2.columns=t2

c2.sort_values(by=t2, axis=0, inplace=True)

c2.to_excel(r'C:\Users\Sarah\Documents\GEM\code\GGPT-US-EIA\scratch\eiaComparedShared_' + stamp + '.xlsx')